# RAG 베이스 코드 with 업스테이지 API

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

In [ ]:
from rag_utils import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("test-RAG")

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
import os
import requests
from langchain_upstage import UpstageDocumentParseLoader
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage

In [ ]:
# 단계 1: 문서 로드(Load Documents)

# DP
file_path = "./data/고졸 성인학습자_사회_54page.pdf"
loader = UpstageDocumentParseLoader(file_path)

pages = loader.load()  # or loader.lazy_load()
for page in pages:
    print(page)

In [ ]:
len(pages)

In [ ]:
pages[0].page_content

In [ ]:
# matadata 확인
pages[0].__dict__

In [ ]:
# OCR
api_key = os.environ['UPSTAGE_API_KEY']
filename = "./data/고졸 성인학습자_사회_54page.pdf"

url = "https://api.upstage.ai/v1/document-ai/ocr"
headers = {"Authorization": f"Bearer {api_key}"}

files = {"document": open(filename, "rb")}
response = requests.post(url, headers=headers, files=files)

print(response.json())

In [ ]:
response.json()["text"]

In [ ]:
# OCR을 이용해서 content 수정하기
pages[0].page_content = response.json()["text"]

In [ ]:
pages

In [ ]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
split_documents = text_splitter.split_documents(pages)
print(f"분할된 청크의수: {len(split_documents)}")

In [ ]:
split_documents

In [ ]:
# 단계 3: 임베딩(Embedding) 생성
# embeddings = UpstageEmbeddings()

embeddings = UpstageEmbeddings(model="embedding-query")

In [ ]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [ ]:
vectorstore

In [ ]:
for doc in vectorstore.similarity_search("산업화"):
    print(doc.page_content)
    print("+" * 100)

In [ ]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [ ]:
retriever

In [ ]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("산업화의 의미는?")

In [ ]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [ ]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatUpstage(model="solar-pro")

In [ ]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# 생성된 체인에 쿼리(질문)을 입력하고 실행합니다.

# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "산업화의 의미는?"
response = chain.invoke(question)
print(response)

## RAG 사용 안한 경우

In [ ]:
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Answer in Korean.

#Question:
{question}

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatUpstage(model="solar-pro")

# 단계 8: 체인(Chain) 생성
chain = {"question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "산업화의 의미는?"
response = chain.invoke(question)
print(response)